<a href="https://colab.research.google.com/github/hohohohyun/django/blob/master/%ED%95%9C%EA%B5%AD%EC%96%B4_LDA_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Kkma, Okt

In [3]:
!pip install konlpy

네이버영화 리뷰 데이터를 이용한 토픽 모델링 예시 입니다.

In [51]:
mr = pd.read_csv('/content/drive/My Drive/movie_review.txt', sep='\t', quoting=3)

In [52]:
mr

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [5]:
## 문장 데이터만 사용
ex_data = mr['document']

In [6]:
#빈 문자열 NAN 값으로 바꾸기
clean_Data = ex_data.replace({'': np.nan})
clean_Data = clean_Data.replace(r'^\s*$', None, regex=True)

#한글이 아닌 단어 공백으로 처리
clean_Data = clean_Data.replace('^ ㄱ-힣', '', regex=True)

#NAN 이 있는 행은 삭제
clean_Data.dropna(how='any', inplace=True)

#행을 제거하면서 망가진 인덱스 초기화
clean_Data = clean_Data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(clean_Data.isnull().values.any()) 

False


In [8]:
from sklearn.decomposition import LatentDirichletAllocation

####형태소 분석기 꼬꼬마 사용



In [32]:
## 불용어 설정은 띄어쓰기로 구분하여 설정하시면 됩니다
## 기존에 설정되어있는 불용어 사전을 사용하려다가 토픽분석을 실시하면서 나오는 불용어들을 추가하면서 만들었습니다. 기존에 존재하는 한국어 불용어사전을 사용하셔도 됩니다.
## 한국어 불용어 사전 링크 https://www.ranks.nl/stopwords/korean

stop_words = '''ㅋㅋ 너무 진짜 정말 ㅎㅎ 같은 그냥 ㅋㅋㅋ 진짜 역시 그럼 근데 이거 하세요 ㅡㅡ ㅋㅋㅋㅋ ㅠㅠ 에서 한테 으로 하고 인데 원래 인가 이나 ㅉㅉ 하는 이다 하냐 해도 이랑 했는데 건가 합시다 하나 에는 하네 보고 보다 입니다 인지 이지 하면
이건 ㅋㅋㅋㅋㅋ 되는 이네 하지 이고 40 봤습니다 10 하다 이건 봤는데 까지 던데 ㄴ데 내가 우리'''

stop_words=stop_words.split(' ') ## 띄어쓰기로 구분된 불용어 리스트로 재설정(하나씩 따옴표 써가면서 만들기가 번거로워서 선택한 방법입니다.)

In [48]:
from konlpy.tag import Kkma  

## Kkma를 이용한 한국어 토큰화 과정

def tokenizing(tag, data):
  ## 선택한 품사tag 방식에 따른 토큰화 과정
  tag_=tag()
  tk_data = []
  for tkns in data:  
    tk_data.append(tag_.morphs(tkns))

  ## 불용어 제거단계  
  result = [] 
  for tkns in tk_data:
    tmp = []
    for w in tkns: 
      if w not in stop_words: 
          tmp.append(w) 
    result.append(tmp)
  
  return result

result = tokenizing(Kkma, clean_Data[-3000:]) ## 3000개로 예시

명사만 사용한 방법과 모든 단어를 다 사용한 방법을 둘다 사용하기 위해서 함수를 하나 더 설정했습니다.

In [34]:
## TF-IDF 행렬을 만들기 위하여 다시 띄어쓰기로 구분하여 문장형식으로 재구성
def corpus(tokens):
  corp = []
  for sent in tokens:
    tmp = " ".join(sent)
    corp.append(tmp)

  return corp

train_corp = corpus(result)

In [35]:
## 키워드 분석을 위하여 명사들만 추출
def n_corpus(corpus):
  corp = []
  for sent in corpus:
    tmp = " ".join(kkma.nouns(sent)) # Kkma에서 제공하는 품사 태깅에서 명사들만 추출하여 문장을 재구성한다.
    corp.append(tmp)
  return corp

n_corp = n_corpus(train_corp)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

## 명사 뿐이 아닌 다른 단어도 사용하기 위해서는 n_corp 대신 train_corp 사용
tfidfv = TfidfVectorizer().fit(n_corp)
tfidf_km = tfidfv.transform(n_corp).toarray()
print(tfidfv.vocabulary_) ## 구성단어 사전

{'파일': 3783, '이병': 2842, '시사회': 2102, '어용': 2366, '리얼리티': 1045, '교육': 411, '보이': 1516, '영화': 2534, '때문': 950, '케이블': 3644, '엄마': 2392, '드라마': 911, '리하': 1054, '궁금하': 456, '금하': 509, '가관': 90, '에피소드': 2416, '이후': 2894, '감명깊': 147, '명깊': 1194, '무엇': 1275, '주고': 3295, '주인공': 3320, '공허': 366, '내용': 652, '진부': 3415, '액션': 2323, '재밌': 3072, '손오공': 1961, '한자': 3975, '마법': 1075, '추억': 3574, '명작': 1202, '신데렐라': 2138, '지금': 3369, '재미있': 3068, '미있': 1335, '터널': 3712, '마지막': 1095, '모습': 1217, '여자': 2446, '재미': 3066, '시간': 2076, '이것': 2814, '50': 51, '년대': 667, '대박': 786, '범죄': 1472, '귀신': 460, '반전': 1400, '악의': 2274, '짬뽕': 3453, '종결': 3277, '눈물': 699, '억지': 2381, '누나': 693, '22': 34, '될려': 899, '개소리': 194, '마라': 1065, '에르몬': 2407, '에르몬넬': 2408, '연기력': 2476, '돋보': 861, '오페라': 2593, '몰입': 1243, '쓰레기': 2205, '옹이': 2606, '배우': 1446, '느낌': 711, '어필': 2379, '막장': 1107, '생각': 1800, '중간': 3343, '후반부': 4132, '개성': 193, '조연': 3252, '활동': 4114, '허무': 4028, '이상한': 2849, '작전': 3022, '에디머': 2404, '피의': 3892, '맨

In [37]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,learning_method='online',random_state=0,max_iter=1)

In [38]:
lda_top=lda_model.fit_transform(tfidf_km)

In [39]:
terms = tfidfv.get_feature_names() ## TF-IDF vectorizer 안에 있는 단어들

def get_topics(components, feature_names, n=10): ## n : 토픽 별로 나타나는 단어의 갯수
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

## 결과 도출(Kkma)
get_topics(lda_model.components_,terms)

Topic 1: [('영화', 39.31), ('이것', 25.66), ('스토리', 22.23), ('생각', 20.07), ('괜찮', 19.29), ('쓰레기', 18.81), ('무엇', 16.06), ('드라마', 15.83), ('공포', 13.65), ('액션', 13.18)]
Topic 2: [('재미있', 32.72), ('미있', 32.71), ('영화', 24.29), ('배우', 17.94), ('인생', 14.73), ('장면', 13.71), ('극장', 13.15), ('감동적', 11.28), ('옛날', 10.83), ('연기', 10.72)]
Topic 3: [('감독', 11.25), ('귀엽', 9.82), ('필요', 6.01), ('영화', 5.75), ('만점', 4.96), ('화려', 4.67), ('인용', 4.66), ('조금', 4.41), ('대체', 4.17), ('표현', 3.97)]
Topic 4: [('재밌', 76.49), ('최고', 48.79), ('영화', 34.8), ('재미', 30.29), ('사랑', 17.91), ('사람', 17.86), ('이야기', 13.4), ('개봉', 11.2), ('시간', 10.49), ('웃기', 10.38)]
Topic 5: [('영화', 93.94), ('평점', 39.46), ('재미없', 29.23), ('미없', 29.07), ('최악', 24.74), ('연기', 23.64), ('명작', 22.13), ('기대', 21.87), ('완전', 19.15), ('시간', 17.02)]


#### 형태소 분석기 Okt(Open Korea Text) 사용
 - 위와 동일하고 형태소 분석기만 Okt를 사용하였습니다.

In [40]:
from konlpy.tag import Okt  

result_o = tokenizing(Okt, clean_Data[-3000:]) ## 3000개로 예시

In [41]:
train_corp_o = corpus(result_o)

In [42]:
n_corp_o = n_corpus(train_corp_o)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(n_corp_o)
tfidf_ = tfidfv.transform(n_corp_o).toarray()
print(tfidfv.vocabulary_)

{'파일': 3856, '이병': 2845, '시사회': 2101, '리얼리티': 1051, '교육': 334, '영화인': 2532, '봣습니당': 1601, '고잉홈': 249, '때문': 933, '케이블': 3708, '엄마': 2396, '드라마': 895, '영환': 2534, '영화니': 2529, '가관': 1, '에피소드': 2417, '쵸파': 3625, '이후': 2911, '가장': 29, '감명': 64, '뭔가': 1328, '보기': 1551, '주인공': 3373, '내용': 619, '액션': 2318, '손오공': 1970, '한자': 4046, '마법': 1077, '추억': 3631, '명작': 1223, '신데렐라': 2130, '지금': 3426, '터널': 3779, '김희라': 516, '마지막': 1098, '모습': 1240, '여자': 2455, '재미': 3100, '시간': 2080, '영화': 2525, '대박': 762, '범죄영화': 1516, '도아': 837, '고귀': 220, '신영화': 2143, '최악': 3616, '짬뽕': 3502, '종결': 3336, '눈물': 665, '억지로': 2383, '졸리': 3329, '누나': 661, '전부': 3162, '소리': 1942, '절대': 3183, '이영화': 2876, '에르': 2407, '몬넬': 1261, '야호': 2333, '연기력': 2480, '오페라': 2591, '몰입': 1265, '쓰레기': 2198, '옹이': 2610, '배우': 1484, '느낌': 678, '병맛': 1547, '어필': 2378, '막장드라마': 1117, '로만': 1006, '생각': 1823, '중간': 3394, '부터': 1628, '점점': 3201, '후반': 4239, '개성': 115, '절떄': 3186, '조연': 3299, '활동': 4218, '작전': 3046, '다가': 692, '그게': 400, '노답': 63

In [44]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,learning_method='online',random_state=0,max_iter=1)

In [45]:
lda_top=lda_model.fit_transform(tfidf_)

In [46]:
terms = tfidfv.get_feature_names() ## TF-IDF vectorizer 안에 있는 단어들

def get_topics(components, feature_names, n=10): ## n : 토픽 별로 나타나는 단어의 갯수
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

## 결과 도출(Okt)
get_topics(lda_model.components_,terms)

Topic 1: [('최고', 42.07), ('영화', 15.97), ('지금', 13.09), ('나름', 7.49), ('아주', 7.41), ('다시', 7.3), ('꿀잼', 7.01), ('배우', 6.23), ('노래', 5.99), ('기억', 5.88)]
Topic 2: [('평점', 42.94), ('연기', 19.12), ('명작', 17.94), ('대박', 13.33), ('이영화', 13.14), ('영화', 11.38), ('사랑', 11.07), ('얼굴', 7.46), ('음악', 7.45), ('소리', 5.99)]
Topic 3: [('영화', 125.8), ('사람', 30.9), ('처음', 18.56), ('스토리', 17.5), ('생각', 13.97), ('소재', 13.15), ('때문', 12.21), ('극장', 12.16), ('옛날', 11.79), ('다시', 11.61)]
Topic 4: [('영화', 39.32), ('재미', 31.18), ('시간', 27.67), ('드라마', 22.25), ('쓰레기', 19.02), ('별로', 18.76), ('작품', 17.53), ('액션', 16.6), ('한번', 15.38), ('최악', 15.07)]
Topic 5: [('감동', 21.52), ('영화', 18.37), ('완전', 17.12), ('개봉', 14.26), ('이야기', 14.12), ('생각', 13.78), ('가슴', 10.3), ('스토리', 10.26), ('눈물', 9.77), ('기대', 9.36)]
